In [28]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from xfeat import Pipeline, SelectCategorical, LambdaEncoder
from xfeat.num_encoder import SelectNumerical




In [29]:
all_data = pd.read_csv('../data/all_data.csv')

In [30]:
all_data.head(3)

,id,region,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state,price
0,0,nashville,1949,bmw,excellent,6 cylinders,gas,115148,clean,manual,rwd,mid-size,convertible,orange,NaN,27587.0
1,1,state college,2013,toyota,fair,8 cylinders,gas,172038,clean,automatic,rwd,full-size,sedan,silver,pa,4724.0
2,2,wichita,1998,ford,good,6 cylinders,gas,152492,clean,automatic,fwd,full-size,SUV,silver,ks,10931.0


reagion

In [31]:
# 頻度が高い地域のリスト
common_regions = ['central NJ', 'rhode island', 'rochester', 'albany', 'washington, DC', 'las vegas', 'tampa bay area', 'los angeles', 'richmond', 'dallas / fort worth']

# 新しいカラムを作成
all_data['region_encoded'] = all_data['region'].apply(lambda x: x if x in common_regions else 'Other values')

year

In [32]:
# yearカラムの中央値を計算
median_year = all_data["year"].median()

# 欠損値を中央値で埋める
all_data["year"].fillna(median_year, inplace=True)

# 2025年以上の値を抜き出して1000引く
all_data.loc[all_data["year"] >= 2025, "year"] -= 1000

# 製造されてからたった年数を計算して新しいカラムを作成
all_data["years_since_manufacture"] = 2023 - all_data["year"]


manufacture

In [33]:
# 頻度が高いメーカーのリスト
common_manufacturers = ['ford', 'chevrolet', 'bmw', 'honda', 'toyota', 'ram', 'jeep', 'nissan', 'volkswagen', 'FORD']

# メーカー名を小文字に統一
all_data['manufacturer'] = all_data['manufacturer'].str.lower()

# 新しいカラムを作成
all_data['manufacturer_encoded'] = all_data['manufacturer'].apply(lambda x: x if x in common_manufacturers else 'Other values')


condition

In [34]:
# 条件と対応する数値のマッピングを定義
condition_mapping = {
    "salvage": 1,
    "fair": 2,
    "good": 3,
    "excellent": 4,
    "like new": 5,
    "new": 6
}

# conditionカラムの値をマッピングに基づいて数値に変換
all_data["condition_encoded"] = all_data["condition"].map(condition_mapping)


cylinders

In [35]:
# 数字だけを抽出
all_data['cylinders_encoded'] = all_data['cylinders'].str.extract('(\d+)')

# NaNを処理
all_data['cylinders_encoded'].fillna(-1, inplace=True)  # 一時的に-1でNaNを置き換え

# 整数型に変換
all_data['cylinders_encoded'] = all_data['cylinders_encoded'].astype(int)

fuel

In [36]:
# 新しいカラムを作成
all_data['fuel_encoded'] = all_data['fuel'].apply(lambda x: x if x in ['gas', 'diesel'] else 'Other Values')

odometer

In [37]:
# 五分位数を計算
q20 = all_data['odometer'].quantile(0.20)
q40 = all_data['odometer'].quantile(0.40)
q60 = all_data['odometer'].quantile(0.60)
q80 = all_data['odometer'].quantile(0.80)

# 新しいカラムを作成
def categorize_mileage_5(value):
    if value == -1:
        return 'Other Values'
    elif value <= q20:
        return 'very_low_mileage'
    elif q20 < value <= q40:
        return 'low_mileage'
    elif q40 < value <= q60:
        return 'medium_mileage'
    elif q60 < value <= q80:
        return 'high_mileage'
    else:
        return 'very_high_mileage'

all_data['mileage_encoded'] = all_data['odometer'].apply(categorize_mileage_5)



tile_status

In [38]:
# 新しいカラムを作成
def categorize_title_status(value):
    top_4_statuses = ["clean", "rebuilt", "salvage", "lien"]
    if value in top_4_statuses:
        return value
    else:
        return 'Other Values'

all_data['title_status_encoded'] = all_data['title_status'].apply(categorize_title_status)


transmission

drive

size

type

paint_color

state

price

平均走行距離/年を計算

In [39]:
all_data['avg_mileage_per_year'] = all_data['odometer'] / all_data['years_since_manufacture']


manufacturer を使用して、各メーカーの中古車の出品数をカウントし、メーカーの人気度を示す

In [40]:
# 各メーカーの中古車の出品数をカウント
manufacturer_counts = all_data['manufacturer'].value_counts()

# 新しいカラム 'manufacturer_popularity' を作成して、出品数を代入
all_data['manufacturer_popularity'] = all_data['manufacturer'].map(manufacturer_counts)


In [41]:
# 各地域の中古車の平均価格を計算
region_avg_price = all_data.groupby('region')['price'].mean()

# 新しいカラム 'region_avg_price' を作成して、平均価格を代入
all_data['region_avg_price'] = all_data['region'].map(region_avg_price)

In [42]:
# size と type を組み合わせて新しいカテゴリ特徴量 'size_type' を作成
all_data['size_type'] = all_data['size'] + "_" + all_data['type']



In [43]:
# drive と transmission を組み合わせて新しいカテゴリ特徴量 'drive_transmission' を作成
all_data['drive_transmission'] = all_data['drive'] + "_" + all_data['transmission']

In [44]:
# paint_color ごとに出品数をカウント
color_popularity = all_data['paint_color'].value_counts()

# 新しい特徴量 'color_popularity_score' を作成
all_data['color_popularity_score'] = all_data['paint_color'].map(color_popularity)


In [45]:
# 暖色系と寒色系の色を定義
warm_colors = ['red', 'orange', 'brown', 'custom']
cold_colors = ['black', 'silver', 'blue', 'white', 'green', 'grey']

# 新しい特徴量 'color_temperature' を作成
def categorize_color_temperature(color):
    if color in warm_colors:
        return 'warm'
    elif color in cold_colors:
        return 'cold'
    else:
        return 'unknown'

all_data['color_temperature'] = all_data['paint_color'].apply(categorize_color_temperature)


In [46]:
# 状態と年齢の組み合わせの特徴量を作成
all_data['condition_age'] = all_data['condition'] + "_" + ((all_data['years_since_manufacture'] // 10) * 10).astype(str) + "years"

In [47]:
# メーカーと国のマッピングを作成
manufacturer_country_mapping = {
    'ford': 'American',
    'chevrolet': 'American',
    'bmw': 'German',
    'honda': 'Japanese',
    'toyota': 'Japanese',
    'ram': 'American',
    'jeep': 'American',
    'nissan': 'Japanese',
    'volkswagen': 'German',
    'FORD': 'American'  # FORDは大文字で指定されているため、別途マッピングを追加
}

# 新しい特徴量を作成
all_data['manufacturer_country'] = all_data['manufacturer'].map(manufacturer_country_mapping)


In [48]:
#連続量を抜き出す
continuous_columns = all_data.select_dtypes(exclude=['object']).columns.tolist()
print(continuous_columns)
# 連続量のデータフレーム
all_data_continuous = pd.DataFrame(all_data[continuous_columns])

all_data_continuous.head()

['id', 'year', 'odometer', 'price', 'years_since_manufacture', 'condition_encoded', 'cylinders_encoded', 'avg_mileage_per_year', 'manufacturer_popularity', 'region_avg_price', 'color_popularity_score']


,id,year,odometer,price,years_since_manufacture,condition_encoded,cylinders_encoded,avg_mileage_per_year,manufacturer_popularity,region_avg_price,color_popularity_score
0,0,1949,115148,27587.0,74,4,6,1556.054054,6146,14888.131004,269
1,1,2013,172038,4724.0,10,2,8,17203.800000,3471,7990.000000,8669
2,2,1998,152492,10931.0,25,3,6,6099.680000,13684,12183.523605,8669
3,3,2014,104118,16553.0,9,4,4,11568.666667,13684,15586.365079,7623
4,4,2005,144554,5158.0,18,4,6,8030.777778,13684,10021.908497,2832


In [49]:
# 標準化したいカラムを指定
columns_to_standardize = ['year', 'odometer', 'years_since_manufacture', 'avg_mileage_per_year', 'manufacturer_popularity', 'region_avg_price', 'color_popularity_score']

# 標準化
scaler = StandardScaler()
all_data[columns_to_standardize] = scaler.fit_transform(all_data[columns_to_standardize])


In [50]:
# LabelEncoderのインスタンスを作成
label_encoder = LabelEncoder()

# カテゴリ変数のリスト
columns_to_encode = ['transmission', 'drive', 'size', 'type', 'paint_color', 'state', 'region_encoded', 'manufacturer_encoded', 'fuel_encoded', 'mileage_encoded', 'title_status_encoded', 'size_type', 'drive_transmission', 'color_temperature', 'condition_age', 'manufacturer_country']

# 各カテゴリ変数をラベルエンコーディング
label_encoders = {}  # 各カテゴリ変数のエンコーダを保存するための辞書
for column in columns_to_encode:
    le = LabelEncoder()
    all_data[column] = le.fit_transform(all_data[column].astype(str))
    label_encoders[column] = le

In [51]:
# カテゴリ変数を抜き出す
categorical_columns = all_data.select_dtypes(include=['object']).columns.tolist()
print(categorical_columns)
# カテゴリ変数のデータフレーム
all_data_categorical = all_data[categorical_columns]

all_data_categorical.head()


['region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status']


,region,manufacturer,condition,cylinders,fuel,title_status
0,nashville,bmw,excellent,6 cylinders,gas,clean
1,state college,toyota,fair,8 cylinders,gas,clean
2,wichita,ford,good,6 cylinders,gas,clean
3,albany,ford,excellent,4 cylinders,gas,clean
4,redding,ford,excellent,6 cylinders,gas,clean


In [52]:
all_data = all_data[['condition_encoded', 'cylinders_encoded', 'transmission', 'drive', 'size', 'type', 'paint_color', 'state', 'region_encoded', 'manufacturer_encoded', 'fuel_encoded', 'mileage_encoded', 'title_status_encoded', 'size_type', 'drive_transmission', 'color_temperature', 'condition_age', 'manufacturer_country', 'year', 'odometer', 'price', 'years_since_manufacture', 'avg_mileage_per_year', 'manufacturer_popularity', 'region_avg_price', 'color_popularity_score']]

In [53]:
print(all_data.columns)
all_data.head()


Index(['condition_encoded', 'cylinders_encoded', 'transmission', 'drive',
       'size', 'type', 'paint_color', 'state', 'region_encoded',
       'manufacturer_encoded', 'fuel_encoded', 'mileage_encoded',
       'title_status_encoded', 'size_type', 'drive_transmission',
       'color_temperature', 'condition_age', 'manufacturer_country', 'year',
       'odometer', 'price', 'years_since_manufacture', 'avg_mileage_per_year',
       'manufacturer_popularity', 'region_avg_price',
       'color_popularity_score'],
      dtype='object')


,condition_encoded,cylinders_encoded,transmission,drive,size,type,paint_color,state,region_encoded,manufacturer_encoded,...,condition_age,manufacturer_country,year,odometer,price,years_since_manufacture,avg_mileage_per_year,manufacturer_popularity,region_avg_price,color_popularity_score
0,4,6,1,2,4,2,6,27,0,1,...,8,1,-5.814645,-0.018732,27587.0,5.814645,-1.230066,0.010815,0.517514,-1.608824
1,2,8,0,2,1,10,9,39,0,8,...,13,2,0.524050,0.901244,4724.0,-0.524050,1.220846,-0.536761,-1.957131,-0.400558
2,3,6,0,1,1,0,9,16,0,3,...,25,0,-0.961582,0.585163,10931.0,0.961582,-0.518396,1.553852,-0.452741,-0.400558
3,4,4,1,1,4,0,1,35,1,3,...,0,0,0.623092,-0.197099,16553.0,-0.623092,0.338213,1.553852,0.767999,-0.551016
4,4,6,1,1,4,10,8,4,0,3,...,2,0,-0.268287,0.456797,5158.0,0.268287,-0.215928,1.553852,-1.228201,-1.240159


In [54]:
all_data.to_csv('../data_processed/all_data1.csv', index=False)